In [47]:
import yfinance as yf
import os

# 获取数据并清理索引
fund_code = "512890"
start_date = "2020-01-01"
end_date = "2025-02-18"

data = yf.download(
    f"{fund_code}.SS",
    start=start_date,
    end=end_date
)
data.columns = data.columns.droplevel(level='Ticker')
data = data.rename(columns={
    'Open': '开盘价',
    'High': '最高价',
    'Low': '最低价',
    'Close': '收盘价',
    'Volume': '成交量'
})

[*********************100%***********************]  1 of 1 completed


In [48]:
# 保存为Excel（无索引列）
save_path = r"C:\Users\vincent.fei\Downloads"  # ← 自定义路径示例
file_name = f"fund_{fund_code}_{start_date}_{end_date}_yahoo.xlsx"
full_path = os.path.join(save_path, file_name)

data.to_excel(
    full_path,
    index=False,  # 不保存索引
    engine='openpyxl'
)

print("生成文件内容预览：")
print(data.head(3))

生成文件内容预览：
Price          收盘价     最高价     最低价     开盘价       成交量
Date                                                
2020-01-02  0.6140  0.6170  0.6025  0.6025  15377000
2020-01-03  0.6115  0.6135  0.6105  0.6135  14113600
2020-01-06  0.6110  0.6160  0.6090  0.6135  24264200


In [49]:
# 定义本金
principal = 1000

# 定义每周定投函数
def simulate_investment(values_daily, frequency):
    # 按周重采样并取每周最后一个交易日的数据
    weekly_data = values_daily.resample(frequency).last().dropna()
    # 计算购买份额
    weekly_data['购买份额'] = principal / weekly_data['收盘价']
    # 计算总份额
    total_shares = weekly_data['购买份额'].sum()
    # 计算最终资产价值
    final_value = total_shares * weekly_data['收盘价'].iloc[-1]
    # 计算投资期（年）
    years = (weekly_data.index[-1] - weekly_data.index[0]).days / 365
    # 计算年化收益率
    if years == 0:
        annual_return = 0
    else:
        annual_return = (final_value / (principal * len(weekly_data))) ** (1 / years) - 1
    return annual_return


# 计算每周定投的年化收益率
weekly_return = simulate_investment(data, 'W')
# 计算每半月定投的年化收益率
biweekly_return = simulate_investment(data, 'SME')
# 计算每月定投的年化收益率
monthly_return = simulate_investment(data, 'ME')

# 输出结果
print(f'每周定投的年化收益率：{weekly_return * 100:.2f}%')
print(f'每半月定投的年化收益率：{biweekly_return * 100:.2f}%')
print(f'每月定投的年化收益率：{monthly_return * 100:.2f}%')

每周定投的年化收益率：6.12%
每半月定投的年化收益率：6.12%
每月定投的年化收益率：6.16%
